# Temporal Occupancy Calculation Notebook

Find out which single cell heatmaps actuallly occupy the temporal range stated on the graphs

In [56]:
import os, re
import tifffile as tiff
from tqdm import tqdm

In [3]:
single_cell_dir = '/home/nathan/data/kraken/h2b/giulia/radial_analysis_output/individual_spatiotemporal_heatmaps/800,20/scr_apop_wt_div'

In [65]:
apop_IDs = [i[:-16].split('_') for i in os.listdir(single_cell_dir) if 'N_cells' in i]
apop_IDs

[['GV0804', 'Pos4', '179'],
 ['GV0796', 'Pos8', '31'],
 ['GV0796', 'Pos5', '1510'],
 ['GV0795', 'Pos4', '359'],
 ['GV0814', 'Pos11', '175'],
 ['GV0798', 'Pos7', '91'],
 ['GV0817', 'Pos12', '975'],
 ['GV0806', 'Pos4', '242'],
 ['GV0815', 'Pos23', '157'],
 ['GV0797', 'Pos7', '204'],
 ['GV0820', 'Pos5', '38'],
 ['GV0814', 'Pos0', '22'],
 ['GV0819', 'Pos1', '503'],
 ['GV0795', 'Pos6', '184'],
 ['GV0797', 'Pos23', '125'],
 ['GV0801', 'Pos5', '1030'],
 ['GV0819', 'Pos5', '206'],
 ['GV0804', 'Pos5', '918'],
 ['GV0796', 'Pos5', '527'],
 ['GV0795', 'Pos6', '736'],
 ['GV0795', 'Pos4', '487'],
 ['GV0801', 'Pos5', '1709'],
 ['GV0815', 'Pos12', '69'],
 ['GV0818', 'Pos12', '113'],
 ['GV0795', 'Pos18', '802'],
 ['GV0797', 'Pos4', '36'],
 ['GV0817', 'Pos23', '472'],
 ['GV0819', 'Pos3', '887'],
 ['GV0820', 'Pos3', '746'],
 ['GV0795', 'Pos18', '28'],
 ['GV0801', 'Pos5', '2306'],
 ['GV0796', 'Pos24', '56'],
 ['GV0818', 'Pos12', '263'],
 ['GV0798', 'Pos20', '239'],
 ['GV0796', 'Pos3', '82'],
 ['GV0796', '

In [35]:
movie_IDs = set([i[0]+'_'+i[1] for i in apop_IDs])
movie_IDs

{'GV0794_Pos4',
 'GV0794_Pos5',
 'GV0794_Pos6',
 'GV0794_Pos7',
 'GV0794_Pos8',
 'GV0795_Pos16',
 'GV0795_Pos17',
 'GV0795_Pos18',
 'GV0795_Pos3',
 'GV0795_Pos4',
 'GV0795_Pos5',
 'GV0795_Pos6',
 'GV0795_Pos7',
 'GV0795_Pos8',
 'GV0796_Pos19',
 'GV0796_Pos20',
 'GV0796_Pos21',
 'GV0796_Pos22',
 'GV0796_Pos23',
 'GV0796_Pos24',
 'GV0796_Pos3',
 'GV0796_Pos4',
 'GV0796_Pos5',
 'GV0796_Pos6',
 'GV0796_Pos7',
 'GV0796_Pos8',
 'GV0797_Pos18',
 'GV0797_Pos19',
 'GV0797_Pos20',
 'GV0797_Pos21',
 'GV0797_Pos22',
 'GV0797_Pos23',
 'GV0797_Pos3',
 'GV0797_Pos4',
 'GV0797_Pos5',
 'GV0797_Pos6',
 'GV0797_Pos7',
 'GV0797_Pos8',
 'GV0798_Pos18',
 'GV0798_Pos20',
 'GV0798_Pos21',
 'GV0798_Pos22',
 'GV0798_Pos23',
 'GV0798_Pos4',
 'GV0798_Pos5',
 'GV0798_Pos6',
 'GV0798_Pos7',
 'GV0798_Pos8',
 'GV0800_Pos15',
 'GV0800_Pos16',
 'GV0800_Pos17',
 'GV0800_Pos3',
 'GV0800_Pos4',
 'GV0800_Pos5',
 'GV0800_Pos6',
 'GV0800_Pos7',
 'GV0800_Pos8',
 'GV0801_Pos3',
 'GV0801_Pos4',
 'GV0801_Pos5',
 'GV0802_Pos3',
 

I have just realised i need every movie (100 movies!) to do this analysis so will scale over movies i have first with os.path.exists()

In [57]:
mov_time_range = {}
missing_movies = []
parent_dir = '/media/nathan/Transcend/Nathan/data/h2b/'
for movie_ID in tqdm(movie_IDs):
    movie_dir = os.path.join(movie_ID.split('_')[0],movie_ID.split('_')[1])
    movie_path = os.path.join(parent_dir, movie_dir, 'stacks')
    if os.path.exists(movie_path):
        movie_fn = os.listdir(movie_path)[0]
        time_range = tiff.imread(os.path.join(movie_path, movie_fn)).shape[0]
        mov_time_range[movie_ID] = time_range
    else:
        missing_movies.append(movie_ID)
    

100%|██████████| 110/110 [09:26<00:00,  5.15s/it]


In [58]:
mov_time_range

{'GV0798_Pos4': 961,
 'GV0794_Pos7': 953,
 'GV0795_Pos4': 1184,
 'GV0794_Pos4': 956,
 'GV0806_Pos4': 1179,
 'GV0804_Pos4': 1162,
 'GV0797_Pos4': 682,
 'GV0800_Pos4': 1189,
 'GV0795_Pos17': 1157,
 'GV0800_Pos16': 1192,
 'GV0802_Pos4': 1195,
 'GV0798_Pos20': 964,
 'GV0797_Pos20': 674,
 'GV0796_Pos22': 901,
 'GV0805_Pos4': 1193,
 'GV0807_Pos3': 1200,
 'GV0797_Pos7': 672,
 'GV0798_Pos21': 961,
 'GV0797_Pos21': 682,
 'GV0796_Pos4': 1002,
 'GV0798_Pos7': 963,
 'GV0795_Pos7': 1183,
 'GV0796_Pos7': 1002,
 'GV0800_Pos7': 1187,
 'GV0801_Pos4': 1194,
 'GV0796_Pos21': 996,
 'GV0807_Pos4': 1171}

In [64]:
cell_type = 'RFP'
path_to_apop_lists = '/media/nathan/Transcend/Nathan/data/h2b/apoptosis_information/final/all'
expts_apop_lists = os.listdir(path_to_apop_lists)
apop_dict = {}
N_apops = len(expts_apop_lists)
for expt_apop_list in expts_apop_lists:
    apop_list = open(os.path.join(path_to_apop_lists, expt_apop_list), 'r')
    for apop_ID in apop_list:
        if cell_type in apop_ID:
            if 'stitched' not in apop_ID: ## relic of apoptosis finding (stitched = tracks that apoptosis switches into post apop)
                apop_dict[apop_ID.split()[0]] = apop_ID.split()[1]
orig_len_apops = len(apop_dict)
apop_dict

{'GV0804_Pos4_179_RFP': '625',
 'GV0804_Pos4_249_RFP': '850',
 'GV0804_Pos4_110_RFP': '851',
 'GV0804_Pos4_292_RFP': '902',
 'GV0804_Pos4_342_RFP': '999',
 'GV0804_Pos4_343_RFP': '1002',
 'GV0804_Pos4_231_RFP': '1010',
 'GV0804_Pos4_339_RFP': '1009',
 'GV0804_Pos4_333_RFP': '1014',
 'GV0804_Pos4_353_RFP': '1050',
 'GV0804_Pos4_351_RFP': '1084',
 'GV0804_Pos4_163_RFP': '1074',
 'GV0804_Pos4_217_RFP': '1142',
 'GV0794_Pos4_6_RFP': '469',
 'GV0794_Pos4_73_RFP': '773',
 'GV0794_Pos4_70_RFP': '820',
 'GV0794_Pos7_52_RFP': '195',
 'GV0794_Pos7_192_RFP': '258',
 'GV0794_Pos7_352_RFP': '749',
 'GV0794_Pos7_328_RFP': '827',
 'GV0794_Pos7_329_RFP': '846',
 'GV0794_Pos7_360_RFP': '874',
 'GV0794_Pos7_429_RFP': '889',
 'GV0795_Pos4_1_RFP': '6',
 'GV0795_Pos4_7_RFP': '8',
 'GV0795_Pos4_103_RFP': '472',
 'GV0795_Pos4_284_RFP': '639',
 'GV0795_Pos4_235_RFP': '642',
 'GV0795_Pos4_234_RFP': '671',
 'GV0795_Pos4_285_RFP': '654',
 'GV0795_Pos4_223_RFP': '418',
 'GV0795_Pos4_323_RFP': '956',
 'GV0795_Pos4

### Iterating through single cell numpy arrays

In [67]:
apop_ID = apop_IDs[0]

In [69]:
for i,j in enumerate(apop_dict):
    if i<1:
        apop_ID = j
    

In [94]:
heatmap_errors = []
count = 0
for apop_ID in apop_dict:

    apop_time = int(apop_dict[apop_ID])

    movie_ID = apop_ID.split('_')[0]+'_'+apop_ID.split('_')[1]
    if movie_ID in mov_time_range:
        movie_length = mov_time_range[movie_ID]
        count+=1
        if movie_length-apop_time < 800/2:
            print("Movie len=", movie_length, " apop_time=", apop_time, " remaining time after apop=", movie_length-apop_time)
            heatmap_errors.append(apop_ID)
            

Movie len= 1162  apop_time= 850  remaining time after apop= 312
Movie len= 1162  apop_time= 851  remaining time after apop= 311
Movie len= 1162  apop_time= 902  remaining time after apop= 260
Movie len= 1162  apop_time= 999  remaining time after apop= 163
Movie len= 1162  apop_time= 1002  remaining time after apop= 160
Movie len= 1162  apop_time= 1010  remaining time after apop= 152
Movie len= 1162  apop_time= 1009  remaining time after apop= 153
Movie len= 1162  apop_time= 1014  remaining time after apop= 148
Movie len= 1162  apop_time= 1050  remaining time after apop= 112
Movie len= 1162  apop_time= 1084  remaining time after apop= 78
Movie len= 1162  apop_time= 1074  remaining time after apop= 88
Movie len= 1162  apop_time= 1142  remaining time after apop= 20
Movie len= 956  apop_time= 773  remaining time after apop= 183
Movie len= 956  apop_time= 820  remaining time after apop= 136
Movie len= 953  apop_time= 749  remaining time after apop= 204
Movie len= 953  apop_time= 827  remain

In [95]:
count ## ie the total number of heatmaps i have raw data for

469

In [93]:
len(heatmap_errors) ## ie heatmaps that need editing

289

### Missing movies

In [60]:
len(set(missing_movies))

83

In [61]:
missing_movies

['GV0796_Pos20',
 'GV0813_Pos2',
 'GV0796_Pos23',
 'GV0813_Pos5',
 'GV0802_Pos3',
 'GV0796_Pos3',
 'GV0819_Pos4',
 'GV0800_Pos8',
 'GV0795_Pos6',
 'GV0819_Pos1',
 'GV0797_Pos5',
 'GV0794_Pos8',
 'GV0814_Pos12',
 'GV0820_Pos3',
 'GV0800_Pos17',
 'GV0796_Pos19',
 'GV0796_Pos8',
 'GV0815_Pos11',
 'GV0795_Pos8',
 'GV0805_Pos5',
 'GV0795_Pos5',
 'GV0813_Pos0',
 'GV0798_Pos6',
 'GV0816_Pos23',
 'GV0800_Pos3',
 'GV0818_Pos13',
 'GV0798_Pos23',
 'GV0813_Pos1',
 'GV0794_Pos5',
 'GV0820_Pos1',
 'GV0805_Pos3',
 'GV0819_Pos0',
 'GV0797_Pos6',
 'GV0820_Pos5',
 'GV0802_Pos5',
 'GV0816_Pos14',
 'GV0798_Pos5',
 'GV0797_Pos8',
 'GV0797_Pos22',
 'GV0806_Pos3',
 'GV0818_Pos12',
 'GV0797_Pos19',
 'GV0817_Pos12',
 'GV0820_Pos4',
 'GV0820_Pos0',
 'GV0795_Pos18',
 'GV0801_Pos3',
 'GV0819_Pos5',
 'GV0817_Pos23',
 'GV0819_Pos3',
 'GV0814_Pos11',
 'GV0807_Pos5',
 'GV0813_Pos3',
 'GV0800_Pos15',
 'GV0797_Pos18',
 'GV0820_Pos2',
 'GV0814_Pos0',
 'GV0813_Pos4',
 'GV0795_Pos16',
 'GV0798_Pos8',
 'GV0794_Pos6',
 'GV